In [8]:
#------------------------ Header Comment ----------------------#
# File: quick-start.py
# Date: 2024-10-14
# Author: Weizhi Peng
#--------------------------------------------------------------#
# Purpose:
# This script is designed to test an emotion recognition 
# pipeline using EEG data from the SEED-IV dataset step by step. The pipeline 
# involves data preprocessing, model training, and evaluation using 
# 10-fold cross-validation. It employs a convolutional neural network 
# (CCNN) for emotion classification into four categories based on 
# EEG signals.
#
# Main steps of the pipeline:
# 1. Load and preprocess EEG data from SEED-IV dataset.
# 2. Apply 10-fold cross-validation for model evaluation.
# 3. Train a CCNN model for emotion classification.
# 4. Use early stopping and model checkpointing during training.
# 5. Evaluate the model and display test accuracy for each fold.
#
# Requirements:
# - PyTorch
# - PyTorch Lightning
# - TorchEEG library
#
# Notes:
# This script leverages GPU acceleration (CUDA or MPS) when available, 
# falling back to CPU if neither is accessible.
#--------------------------------------------------------------#




from torcheeg import transforms
from torcheeg.transforms.pyg import ToG
from torcheeg.datasets import SEEDIVDataset
from torcheeg.datasets.constants.emotion_recognition.seed_iv import (SEED_IV_ADJACENCY_MATRIX,
                                                   SEED_IV_CHANNEL_LOCATION_DICT)

# test the SEEDIVDataset
dataset = SEEDIVDataset(io_path=f'../data/SEED_IV/seed_iv',
                     root_path='../data/SEED_IV/eeg_raw_data',
                     offline_transform=transforms.Compose([
                          transforms.BandDifferentialEntropy(),
                          transforms.ToGrid(SEED_IV_CHANNEL_LOCATION_DICT)
                      ]),
                     online_transform=transforms.ToTensor(),
                     label_transform=transforms.Compose([
                         transforms.Select('emotion'),
                         transforms.Lambda(lambda x: x),
                     ]),
                     num_worker=4)
(sample,target) = dataset[0]
print("shape of input sample",sample.shape)
# check how many target classes from dataset
target_list = []
for i in range(len(dataset)):
    _, target = dataset[i]
    target_list.append(target)
print("target_list",set(target_list))







[2024-10-15 13:58:07] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from ../data/SEED_IV/seed_iv.


shape of input sample torch.Size([4, 9, 9])
target_list {0, 1, 2, 3}


In [10]:
# raw data
# from torcheeg.datasets import SEEDIVDataset
# from torcheeg import transforms

# dataset = SEEDIVDataset(root_path='../data/SEED_IV/eeg_raw_data',
#                         io_path='.torcheeg/datasets_1728969147724_1jRHe',
#                         online_transform=transforms.Compose([
#                             transforms.ToTensor(),
#                             transforms.To2d()
#                         ]),
#                         label_transform=transforms.Select('emotion'))

# (sample,target) = dataset[0]
# print("shape of input sample",sample.shape)
# # check how many target classes from dataset
# target_list = []
# for i in range(len(dataset)):
#     _, target = dataset[i]
#     target_list.append(target)
# print("target_list",set(target_list))


In [11]:
from torcheeg.model_selection import KFoldGroupbyTrial

# 10-fold cross-validation
k_fold = KFoldGroupbyTrial(n_splits=10,
                           split_path='./tmp_out/examples_pipeline/split',
                           shuffle=True,
                           random_state=42)

In [12]:
from torch.utils.data import DataLoader
from torcheeg.models import CCNN

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl
import torch



# get accelerator
if torch.cuda.is_available():
    print('cuda is available')
    device = 'cuda'
elif torch.has_mps:
    print('mps is available')
    device = 'mps'
else:
    print('cuda is not available, use cpu')
    device = 'cpu'




cuda is available


In [13]:
# train and test
batch_size = 256
for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    # create data loader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # create model
    model = CCNN(num_classes=4, in_channels=4, grid_size=(9, 9))

    # create trainer
    trainer = ClassifierTrainer(model=model,
                                num_classes=4,
                                lr=1e-4,
                                weight_decay=1e-4,
                                accelerator=device)
    # train and test
    trainer.fit(train_loader,
                val_loader,
                max_epochs=1,
                default_root_dir=f'./tmp_out/examples_pipeline/model/{i}',
                callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
                enable_progress_bar=True,
                enable_model_summary=True,
                limit_val_batches=0.0)
    
    # test
    score = trainer.test(val_loader,
                         enable_progress_bar=True,
                         enable_model_summary=True)[0]
    print(f'Fold {i} test accuracy: {score["test_accuracy"]:.4f}')

[2024-10-15 13:59:21] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from ./tmp_out/examples_pipeline/split.
[2024-10-15 13:59:21] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/jim/anaconda3/envs/eeg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using

Epoch 0:   2%|▏         | 2/132 [00:00<00:28,  4.62it/s, v_num=21, train_loss=1.380, train_accuracy=0.266]

/home/jim/anaconda3/envs/eeg/lib/python3.9/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 0: 100%|██████████| 132/132 [00:03<00:00, 43.95it/s, v_num=21, train_loss=1.340, train_accuracy=0.381]

[2024-10-15 13:59:25] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.371 train_accuracy: 0.298 

`Trainer.fit` stopped: `max_epochs=1` reached.


...

Epoch 0: 100%|██████████| 133/133 [00:02<00:00, 44.46it/s, v_num=1, train_loss=1.340, train_accuracy=0.344]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jim/anaconda3/envs/eeg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 15/15 [00:00<00:00, 68.27it/s]

[2024-10-15 13:59:58] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.310 test_accuracy: 0.374 



Testing DataLoader 0: 100%|██████████| 15/15 [00:00<00:00, 67.16it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.37361112236976624
        test_loss           1.3104280233383179
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Fold 9 test accuracy: 0.3736
